1.	Identify and describe 2 data quality issues present in the dataset. Briefly propose strategies to address these issues. Document the steps taken and provide a summary of the data quality improvements.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import numpy as np
import unittest
import math
from scipy.stats import ttest_ind
from scipy.stats import shapiro

In [2]:
def create_dateset():
    '''Read dataset and pre-process charge columns to replace missing values by zero and round to 2 places of decimal,
    Arguments - None,
    Return - created dataset as pandas dataframe'''
    input_table = pd.read_excel("Data Insights - Synthetic Dataset.xlsx", sheet_name = "Data Insights - Synthetic Datas")

    # for columns pertaining to charges, round off and replace missing values with zero
    numeric_columns = [ 'AccommodationCharge', 'CCU_Charges', 'ICU_Charge', 'TheatreCharge',  'ProsthesisCharge', 'OtherCharges',
       'BundledCharges']

    for column in numeric_columns:
        input_table.loc[:,column + "_Filled"]=input_table.loc[:,column].astype(float).round(2).fillna(0)

    # analyse and investigate scaling on pharmacy charge
    input_table.loc[:,"PharmacyCharge"]=input_table.loc[:,"PharmacyCharge"].apply(lambda x:str(x).replace("ERROR",'0')).astype(float)
    input_table.loc[:,"PharmacyCharge_Filled"]=input_table.loc[:,"PharmacyCharge"].astype(float).round(2).fillna(0)
    input_table.loc[:,"PharmacyCharge_Scaled"] = input_table.loc[:,"PharmacyCharge_Filled"].apply(lambda x:math.log(x) if x>0.01 else 0)

    return input_table

In [3]:
# create dataset
input_table = create_dateset()

In [4]:
def show_comparison (df, col1, col2, num_bins = 50):
    '''Performing comparison of distribution of columns col1 and col2,
    Also prints summary of missing data filled and summary statistics,
    Arguments - df is pandas dataframe with columns col1 and col2 for comparison, num_bins as number of bins in histogram,
    Return - None'''
    # Create subplots for two histograms
    fig, axes = plt.subplots(1, 2, figsize=(10, 5))

    # Calculate bin edges using numpy's linspace for filled columns
    bin_edges = np.linspace(df[col2].min(), df[col2].max(), num_bins + 1)

    sns.histplot(df[col1], bins=bin_edges, ax=axes[0])  

    hist_data = sns.histplot(df[col2], bins=bin_edges, ax=axes[1])  

    # Extract the counts from the histogram
    counts = hist_data.patches  # Patches are the rectangles that make up the histogram

    # Calculate maximum frequency
    max_frequency = max([patch.get_height() for patch in counts]) *1.1   
    maxvalue = df[col2].max()

    # Set y-axis limits for both subplots
    axes[0].set_ylim([0, max_frequency])  
    axes[0].set_xlim([0, maxvalue]) 
    axes[1].set_ylim([0, max_frequency])  
    axes[1].set_xlim([0, maxvalue]) 
    axes[0].grid(visible=True, which='major', linestyle='--', linewidth=0.7, color='gray')  # Major gridlines
    axes[1].grid(visible=True, which='major', linestyle='--', linewidth=0.7, color='gray')  # Major gridlines

    # Show the plot
    plt.tight_layout()
    plt.show()

    # Before cleaning
    missing_before = df[col1].isnull().sum()

    # After cleaning
    missing_after = df[col2].isnull().sum()

    print("Missing data before:\n", missing_before)
    print("Missing data after:\n", missing_after)

    print("Summary before cleaning")
    print(df[col1].describe())

    print("Summary after cleaning")
    print(df[col2].describe())

In [ ]:
show_comparison(input_table,"AccommodationCharge","AccommodationCharge_Filled")

In [ ]:
show_comparison(input_table,"CCU_Charges","CCU_Charges_Filled")

In [ ]:
show_comparison(input_table,"ICU_Charge","ICU_Charge_Filled")

In [ ]:
show_comparison(input_table,"TheatreCharge","TheatreCharge_Filled")

In [ ]:
show_comparison(input_table,"ProsthesisCharge","ProsthesisCharge_Filled")

In [ ]:
show_comparison(input_table,"OtherCharges","OtherCharges_Filled")

In [ ]:
show_comparison(input_table,"BundledCharges","BundledCharges_Filled")

In [ ]:
show_comparison(input_table,"PharmacyCharge_Filled", "PharmacyCharge_Scaled")

Data Transformation and Analysis 

Q2.	Using the data provided create a feature that could be valuable for analysis or modelling. Explain the rationale behind the feature you created and how they might be useful for analysis.

Q6.	Based on your analysis, identify two strategic insights that could help Ramsay improve hospital operations or patient care. Justify your insights with evidence from your data analysis.

In [13]:
def classify_admission(episode):
    '''classify admission entries based on weeday working hours or otherwise to create categorical variable,
    Arguments: row entry for pandas dataframe,
    Return: Classification flag as Weekday_Workinghour, Weekday_NonWorkinghour or Weekend '''
    if episode['AdmissionDayOfWeek'] in [0,1,2,3,4]:
        if episode['AdmissionHour']>=9 and episode['AdmissionHour']<18:
            return 'Weekday_Workinghour'
        else:
            return 'Weekday_NonWorkinghour'
    else:
        return 'Weekend'

def classify_separation(episode):
    '''classify separation entries based on weeday working hours or otherwise to create categorical variable,
    Arguments: row entry for pandas dataframe,
    Return: Classification flag as Weekday_Workinghour, Weekday_NonWorkinghour or Weekend '''
    if episode['SeparationDayOfWeek'] in [0,1,2,3,4]:
        if episode['SeparationHour']>=9 and episode['SeparationHour']<18:
            return 'Weekday_Workinghour'
        else:
            return 'Weekday_NonWorkinghour'
    else:
        return 'Weekend'

In [14]:
def transform_dataset(input_table):
    '''Function to create new features and transform existing features for analysis
    Arguement: input dataframe
    Return: Transformed dataset as pandas dataframe
    '''
    # convert admission provided id to string
    input_table['AdmissionProviderID'] = input_table['AdmissionProviderID'].astype(str)

    # combine date and time columns to create timestamp
    input_table.loc[:,"AdmissionTimestampStr"] = input_table.loc[:,'AdmissionDate'].astype(str) + " " +  input_table.loc[:,'AdmissionTime'].astype(str)
    input_table.loc[:,"AdmissionTimestamp"] = pd.to_datetime(input_table.loc[:,"AdmissionTimestampStr"])

    input_table.loc[:,"SeparationTimestampStr"] = input_table.loc[:,'SeparationDate'].astype(str) + " " +  input_table.loc[:,'SeparationTime'].astype(str)
    input_table.loc[:,"SeparationTimestamp"] = pd.to_datetime(input_table.loc[:,"SeparationTimestampStr"])

    #calculate length of stay based on admission and separation timestamps
    input_table.loc[:,"LengthofStay"] = input_table.loc[:,"SeparationTimestamp"] - input_table.loc[:,"AdmissionTimestamp"]
    input_table.loc[:,"LengthofStay_hours"] = input_table.loc[:,"LengthofStay"].dt.total_seconds() / 3600 
    input_table.loc[:,"LengthofStay_days"] = input_table.loc[:,"LengthofStay_hours"] / 24

    #calculate total charge excluding pharmacy charges
    input_table.loc[:,"TotalCharges"] =  input_table.loc[:,'AccommodationCharge_Filled']+ input_table.loc[:,'CCU_Charges_Filled']+ input_table.loc[:,'TheatreCharge_Filled']+  input_table.loc[:,'ProsthesisCharge_Filled']+ input_table.loc[:,'OtherCharges_Filled']+ input_table.loc[:,'BundledCharges_Filled']

    #create feature for day, day name, hour, day of week, day of month, month, year
    input_table['AdmissionDayOfWeek'] = input_table['AdmissionTimestamp'].dt.dayofweek
    input_table['AdmissionDayName'] = input_table['AdmissionTimestamp'].dt.day_name()
    input_table['AdmissionHour'] = input_table['AdmissionTimestamp'].dt.hour

    input_table['SeparationDayOfWeek'] = input_table['SeparationTimestamp'].dt.dayofweek
    input_table['SeparationDayName'] = input_table['SeparationTimestamp'].dt.day_name()
    input_table['SeparationHour'] = input_table['SeparationTimestamp'].dt.hour

    input_table['AdmissionDay'] = input_table['AdmissionTimestamp'].dt.day
    input_table['AdmissionMonth'] = input_table['AdmissionTimestamp'].dt.month
    input_table['AdmissionYear'] = input_table['AdmissionTimestamp'].dt.year

    input_table['SeparationDay'] = input_table['SeparationTimestamp'].dt.day
    input_table['SeparationMonth'] = input_table['SeparationTimestamp'].dt.month
    input_table['SeparationYear'] = input_table['SeparationTimestamp'].dt.year

    # Define the order of days
    daysofweek = ['Monday', 'Tuesday','Wednesday', 'Thursday', 'Friday', 'Saturday','Sunday']
    day_dtype = pd.CategoricalDtype(categories=daysofweek, ordered=True)

    # convert day name to categorical variable
    input_table['AdmissionDayName'] = input_table['AdmissionDayName'].astype(day_dtype)
    input_table['SeparationDayName'] = input_table['SeparationDayName'].astype(day_dtype)

    # classification of admission based on weekend, weekday working hours or non-working hours
    input_table['Admission_Classification'] = input_table.apply(lambda x: classify_admission(x), axis=1)
    input_table['Separation_Classification'] = input_table.apply(lambda x: classify_separation(x), axis=1)

    #create binary variable to show if infant weight is provided
    input_table['InfantCase'] = input_table['InfantWeight'].apply(lambda x: 'Yes' if x>0 else 'No')

    #create binary variable to show if mechanical ventilation was used
    input_table['VentilationCase'] = input_table['HoursMechVentilation'].apply(lambda x: 'Yes' if x>0 else 'No')

    return input_table


In [15]:
input_table = transform_dataset(input_table=input_table)

In [16]:
def create_scatter_plot(df,x, y):
    '''Create scatter plot,
    Arguments - dataframe with x and y variables,
    Return - None'''
    # Create a scatter plot
    sns.scatterplot(data=df, x=x, y=y, color='blue')
    plt.xlabel(x)
    plt.ylabel(y)

    # Show the plot
    plt.show()

In [17]:
def create_correlation_matrix(df):
    '''Calculate the correlation matrix and create heatmap,
    Argument - Dataframe with columns for calculation,
    Return - None'''
    correlation_matrix = df.corr()

    # Visualize the correlation matrix using a heatmap
    plt.figure(figsize=(8, 6))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('Correlation Matrix')
    plt.show()

In [18]:
def create_box_plot(df, colx, coly):
    '''Create a box plot using Plotly Express
    Arguments: df is pandas dataframe, colx and coly are x and y columns respectively
    Return: None'''
    fig = px.box(df, x=colx, y=coly, title='Distribution of ' + coly + ' by ' + colx)
    fig.show()

def comparison_summary(df,colx):
    '''Create a summary table of median of length of stay, median of total charge and count of episodes.
    Arguments: df is pandas dataframe, colx and coly are x and y columns respectively
    Return: None'''
    df_by_col = pd.pivot_table(df,index=[colx], aggfunc={'LengthofStay_hours':'median', 'TotalCharges':"median",'episode_id':'count'}).reset_index()
    df_by_col.columns = [colx,"Median_Duration", "Median_Charge","Count_Cases"]
    print(df_by_col)

In [19]:
def plot_mean_comparison(df, colx, coly):
    '''Creates comparison plot for numerical distribution by given columns
    Arguments - dataframe and selected columns
    Returns - None'''
    df_by_col = pd.pivot_table(df,index=[colx], aggfunc={coly:'mean'}).reset_index()
    df_by_col.columns = [colx,"Mean_" + coly]
    fig = px.bar(df_by_col, x=colx, y="Mean_" + coly, text="Mean_" + coly, title = "Distribution of " + coly + ' by '+colx)
    fig.update_traces(textposition='inside')
    fig.update_layout(
    xaxis_title_text=colx,
    yaxis_title_text='Mean of ' + coly)
    fig.show()

def plot_count_comparison(df, colx, coly):
    '''Creates comparison plot of count by given columns
    Arguments - dataframe and selected columns
    Returns - None'''
    df_by_col = pd.pivot_table(df,index=[colx], aggfunc={coly:'count'}).reset_index()
    df_by_col.columns = [colx,"Count_" + coly]
    fig = px.bar(df_by_col, x=colx, y="Count_" + coly, text="Count_" + coly, title = "Distribution of " + coly + ' by '+colx)
    fig.update_traces(textposition='inside')
    fig.update_layout(
    xaxis_title_text=colx,
    yaxis_title_text='Count of ' + coly)
    fig.show()

In [20]:
def compare_plots(df, col):
    '''Create comparison plots and summary for Length of stay, Total charge and number of episodes,
    Arguments - dataframe with column for comparison,
    Return - None'''
    create_box_plot(df, colx = col, coly='LengthofStay_hours')
    create_box_plot(df, colx = col, coly='TotalCharges')
    plot_count_comparison(df, colx = col, coly='episode_id')
    comparison_summary(df,colx = col)

In [ ]:
create_scatter_plot(input_table, x="LengthofStay_hours", y="TotalCharges")

In [ ]:
create_scatter_plot(input_table, x="InfantWeight", y="TotalCharges")
create_scatter_plot(input_table, x="InfantWeight", y="LengthofStay_hours")

In [ ]:
create_scatter_plot(input_table, x="HoursMechVentilation", y="TotalCharges")
create_scatter_plot(input_table, x="HoursMechVentilation", y="LengthofStay_hours")

In [ ]:
create_scatter_plot(input_table, x="Age", y="LengthofStay_hours")
create_scatter_plot(input_table, x="Age", y="TotalCharges")  # no correlation seen

In [ ]:
create_scatter_plot(input_table, x="LengthofStay_hours",y = 'AccommodationCharge_Filled') # indicates a time slab for accommodation charges which changes at 400 hr

In [ ]:
df = input_table.loc[:,["LengthofStay_hours",'AccommodationCharge_Filled', 'CCU_Charges_Filled',
        'ICU_Charge_Filled','TheatreCharge_Filled', 'ProsthesisCharge_Filled','OtherCharges_Filled', 
        'BundledCharges_Filled', 'PharmacyCharge_Scaled', 'InfantWeight','HoursMechVentilation']]

create_correlation_matrix(df)
# correlation seen between accommodation charge and length of stay which is intuitive

In [ ]:
df = input_table.loc[:,["LengthofStay_hours", 'Age']]

create_correlation_matrix(df) # no correlation with age

In [ ]:
compare_plots(input_table, col = 'ModeOfSeparation') # no difference

In [ ]:
compare_plots(input_table, col = 'Sex')  # no difference

In [ ]:
compare_plots(input_table, col = 'insurer_id')  # no difference

In [ ]:
compare_plots(input_table, col = 'CareType')  # no difference

In [ ]:
compare_plots(input_table, col = 'SourceOfReferral')  # no difference

In [ ]:
compare_plots(input_table, col = 'DischargeIntention')  # no difference

In [ ]:
compare_plots(input_table, col = 'AdmissionProviderID')  # no significant difference

In [ ]:
compare_plots(input_table, col = 'UrgencyOfAdmission')  # no difference

In [ ]:
compare_plots(input_table, col = 'UnplannedTheatreVisit')  # slightly higher
# shows a higher duration for unplanned visits which makes sense if emergency services are required

In [37]:
def compare_statistical_summary_yesno(df, colx , coly = ['TotalCharges', 'LengthofStay_hours']):
    '''Compare length of stay and total charges statistically using t-test and Shapiro-Wilk test,
    For t-test p value<0.1 indicates rejection of null hypothesis and shows statistically different mean values in between both groups,
    For Shapiro p value<0.1 indicates rejection of null hypothesis meaning the data is not normally distributed,
    Arguments: dataframe with column for comparison,
    Return : None'''
    df1 = df.loc[df.loc[:,colx]=='Yes',coly]
    df2 = df.loc[df.loc[:,colx]=='No',coly]
    for i in coly:
        print("Comparison for " +  i)
        #t-test
        t_stat, p_value = ttest_ind(df1.loc[:,i], df2.loc[:,i])
        print("T-statistic value: ", t_stat)  
        print("P-Value: ", p_value)
        # shapiro-wilk test to check if sample is normally distributed
        # Print the results
        print("Shapiro-Wilk Test for Group Yes:", shapiro(df1.loc[:,i]))
        # Print the results
        print("Shapiro-Wilk Test for Group No:", shapiro(df2.loc[:,i]))

In [ ]:
compare_plots(input_table, col = 'InfantCase')  # higher charge but smaller length of stay for infant cases

In [ ]:
compare_statistical_summary_yesno(df=input_table, colx = 'InfantCase')

Result shows that for TotalCharges, there is significant difference due to presence of Infant Case. However, ShapiroWilk test indicates data is not normally distributed which is an assumption for validity of t-test
For LengthofStay_hours, there is no significant difference due to presence of Infant Case. However, ShapiroWilk test indicates data is not normally distributed which is an assumption for validity of t-test

In [ ]:
compare_plots(input_table, col = 'Readmission28Days')  # higher charge for Yes value

In [ ]:
compare_statistical_summary_yesno(df=input_table, colx = 'Readmission28Days')

In [ ]:
compare_plots(input_table, col =  'VentilationCase') # no significant difference

In [ ]:
compare_statistical_summary_yesno(input_table, colx = 'VentilationCase')

In [ ]:
compare_plots(input_table, col =  'PalliativeCareStatus')  # higher charge for Palliative care

In [ ]:
compare_statistical_summary_yesno(input_table, colx =  'PalliativeCareStatus')

In [ ]:
compare_plots(input_table, col =  'AdmissionDayOfWeek')  # higher on Monday-Wed

In [ ]:
compare_plots(input_table, col =  'AdmissionDayName') 

In [ ]:
compare_plots(input_table, col =  'SeparationDayName') 

In [ ]:
compare_plots(input_table, col =  'AdmissionDay')  # no significant difference

In [ ]:
compare_plots(input_table, col =  'SeparationDay')  # no significant difference

In [ ]:
compare_plots(input_table, col =  'AdmissionHour')  # no significant difference

In [ ]:
compare_plots(input_table, 'SeparationHour')  # no significant difference

In [ ]:
compare_plots(input_table, 'Admission_Classification')

In [ ]:
compare_plots(input_table, 'Separation_Classification') # no significant difference

3.	Using the data provided produce a piece of analysis that describes to Ramsay which DRGs accrue the largest charges and your hypotheses for the drivers of these charge. Visualise these trends using appropriate charts or graphs and describe the results.

In [55]:
def create_barplot(df, colx, coly, summarytype = 'count', barcount = 20, ascorder = False):
    '''Creates comparison plot of count by given columns
    Arguements - dataframe and selected columns, summarytype is summary required, barcount is number of bars in plot, 
    ascorder is ascending or descending order,
    Returns - None'''
    df_by_col = pd.pivot_table(df,index=[colx], aggfunc={coly:summarytype}).reset_index()
    df_by_col.columns = [colx,summarytype + '_' + coly]
    df_by_col.sort_values([summarytype + '_' + coly], ascending=[ascorder], inplace=True)
    df_by_col = df_by_col.iloc[:barcount,:]
    df_by_col[summarytype + '_' + coly] = df_by_col[summarytype + '_' + coly].round(4)

    fig = px.bar(df_by_col, x=colx, y=summarytype + '_' + coly, text=summarytype + '_' + coly, title = "Distribution of " + coly + ' by '+colx)

    fig.update_traces(textposition='inside')
    fig.update_layout(
    xaxis_title_text=colx,
    yaxis_title_text=summarytype + '_' + coly)
    fig.show()

In [ ]:
create_barplot(input_table, colx = 'AR-DRG', coly = 'TotalCharges', summarytype='mean', ascorder=False, barcount=20)

In [ ]:
create_barplot(input_table, colx = 'AR-DRG', coly = 'TotalCharges', summarytype='median', ascorder=False, barcount=20)

In [ ]:
create_barplot(input_table, colx = 'AR-DRG', coly = 'TotalCharges', summarytype='median', ascorder=True, barcount=20)

In [ ]:
create_barplot(input_table, colx = 'AR-DRG', coly = 'LengthofStay_hours', summarytype='median', ascorder=False, barcount=20)

In [ ]:
create_barplot(input_table, colx = 'AR-DRG', coly = 'LengthofStay_hours', summarytype='median', ascorder=True, barcount=20)

In [ ]:
create_barplot(input_table, colx = 'AR-DRG', coly = 'episode_id', summarytype='count', ascorder=False, barcount=5) # max for DRG001, DRG002, and DRG003

In [62]:
diagnosis_columns = ["PrincipalDiagnosis", "Diagnosis2", "Diagnosis3", "Diagnosis4", "Principal_ProcedureCode",
                      "ProcedureCode2", "ProcedureCode3"]

In [ ]:
for i in diagnosis_columns:
    create_barplot(input_table, colx = i, coly = 'TotalCharges', summarytype='median', ascorder=False, barcount=20)

In [ ]:
for i in diagnosis_columns:
    create_barplot(input_table, colx = i, coly = 'episode_id', summarytype='count', ascorder=False, barcount=20)

Unit Testing for functions

In [ ]:
class TestPlot(unittest.TestCase):
    def test_create_dataset(self):
        """
        Test that the create_dataset function returns a value
        """
        result = create_dateset()
        self.assertIsNotNone(result)

    def test_scatter_plot(self):
        """
        Test that the scatter plot function does not return a value
        """
        result = create_scatter_plot(input_table, x="LengthofStay_hours", y="TotalCharges")
        self.assertIsNone(result)
    
    def test_comparison_summary(self):
        """
        Test that the comparison summary function does not return a value
        """
        result = comparison_summary(input_table, colx='AdmissionDayName')
        self.assertIsNone(result)
    
    
    def test_transform_dateset(self):
        """
        Test that the transform dataset function does return a pandas dataframe
        """
        result = transform_dataset(input_table=input_table)
        self.assertIsInstance(result, pd.DataFrame)

    def test_show_comparison(self):
        """
        Test that the show comparison function does not return a value
        """
        result = show_comparison(input_table,"AccommodationCharge","AccommodationCharge_Filled")
        self.assertIsNone(result)

unittest.main(argv=[''], verbosity=2, exit=False)